In [1]:
import pandas as pd
import numpy as np
import gensim
import nltk
import logging
import pickle
import spacy
import math

import gensim.corpora as corpora

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora, models
from gensim.test.utils import datapath

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from scipy.stats import entropy
from tempfile import TemporaryFile

from scipy.special import (entr, rel_entr)
from numpy import (arange, putmask, ravel, ones, shape, ndarray, zeros, floor,
                   logical_and, log, sqrt, place, argmax, vectorize, asarray,
                   nan, inf, isinf, NINF, empty)

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

np.random.seed(2020)

nltk.download('wordnet')
stemmer = SnowballStemmer('english')

DOC_COUNT = 500000

[nltk_data] Downloading package wordnet to /home/p4l/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
my_stop_words = STOPWORDS.union(set(['use', 'be', 'work', 'user', 'try', 'cell',
                                     'row', 'want', 'item', 'go', 'get', 'add', 'went', 'tried',
                                    'return', 'sort', 'test', 'run', 'check', 'click', 'hour', 'minute', 'second',
                                    'version', 'app', 'paragraph', 'error', 'log', 'press',
                                    'need', 'feed', 'thank', 'way', 'like', 'kill', 'help']))

In [3]:
base_path = "/home/p4l/work/stackoverflow/"
base_model = base_path + "models_data/"
base_dataset = base_path + "dataset/"
base_model_lda = base_model + "lda/"

In [4]:
def m_entropy(pk, qk=None, base=None, axis=0):
    pk = asarray(pk)
    pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)
    if qk is None:
        vec = entr(pk)
    else:
        qk = asarray(qk)
        #if qk.shape != pk.shape:
            #raise ValueError("qk and pk must have same shape.")
        qk = 1.0*qk / np.sum(qk, axis=axis, keepdims=True)
        vec = rel_entr(pk, qk)
    S = np.sum(vec, axis=axis)
    if base is not None:
        S /= log(base)
    return S

def jensen_shannon(query, matrix):
    """
    This function implements a Jensen-Shannon similarity
    between the input query (an LDA topic distribution for a document)
    and the entire corpus of topic distributions.
    It returns an array of length M where M is the number of documents in the corpus
    """
    # lets keep with the p,q notation above
    p = query[None,:].T # take transpose
    q = matrix.T # transpose matrix

    #new = np.zeros((q.shape[0], q.shape[1]))
    #new[:q.shape[0], :1] = p
    #p = new
    
    m = 0.5*(p + q)
    return np.sqrt(0.5*(m_entropy(p,m) + m_entropy(q,m)))

def get_most_similar_documents(query,matrix,k=10):
    """
    This function implements the Jensen-Shannon distance above
    and retruns the top k indices of the smallest jensen shannon distances
    """
    sims = jensen_shannon(query,matrix) # list of jensen shannon distances
    out = [dist for dist in sims if not math.isnan(dist)]
    return sims.argsort()[:k], sorted(sims, reverse=True) # the top k positional index of the smallest Jensen Shannon distances

def clear_text(text):
    text = re.sub('<code>(.|\n)*?<\/code>', '', text)
    text = re.sub(r'(\<(/?[^>]+)>)', '', text)
    text = re.sub("[\'\"\\/\@\%\(\)\~\`\{\}]", '', text)
    text = re.sub('\s+', ' ', text)
    
    return text

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    text = clear_text(text)
    result = []
    #result = [token in gensim.utils.simple_preprocess(text, deacc=True) if ((token not in gensim.parsing.preprocessing.STOPWORDS) and len(token) > 1) == True]
    for token in gensim.utils.simple_preprocess(text, deacc=True):
        if (token not in my_stop_words) and len(token) > 1:
            #result.append(lemmatize_stemming(token))
            result.append(token)
    return result

In [5]:
def split_tags(text):
    if not isinstance(text, str) and math.isnan(text):
        return ''
    if text == '' or text == ' ':
        return text
    else:
        return text.replace('|', ' ')

def add_string(text, tags, n=3):
    tags = split_tags(tags)
    tags = ' ' + tags
    i = 0
    for i in range(n):
        if i % 2 == 0:
            text += tags
        else:
            text = tags + text
    return text

In [6]:
data = pd.read_csv(base_dataset + 'large_data.csv')
if DOC_COUNT != -1:
    data = data.head(DOC_COUNT)

titles = data['post_title'].values
tags = data['post_tags'].map(split_tags).values
data = [add_string(row['post_body'], row['post_tags'], 3) for _, row in data.iterrows()]
#data = data['post_body'].values

[data.append(z) for z in titles]
[data.append(z) for z in tags]
[data.append(z) for z in tags]
[data.append(z) for z in tags]
[data.append(z) for z in tags]

#data = np.array(data)
#data = np.append(data, titles)
#data = np.append(data, tags)
#data = np.append(data, tags)
#data = np.append(data, tags)
#data = np.append(data, tags)

print(len(data))
del titles
del tags

3000000


In [7]:
doc_sample = data[3]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\ntokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['', 'database', 'rdbms', 'ordbms<p>After', 'mounting', 'the', 'database', 'I', 'tried', 'executing', 'these', 'command-', 'alter', 'database', 'open;', 'emanating', 'from', 'a', 'shutdown', 'initialization', 'error', 'but', 'my', 'oracle', 'kept', 'returning', 'an', 'ORA-00600:', 'internal', 'error', 'code,', 'arguments', '[dbkif_find_next_record_1],', '[],', '[],', '[].\nPlease,', 'how', 'do', 'I', 'proceed', 'from', 'here?</p>', 'database', 'rdbms', 'ordbms', 'database', 'rdbms', 'ordbms']


tokenized and lemmatized document: 
['database', 'rdbms', 'ordbmsafter', 'mounting', 'database', 'executing', 'command', 'alter', 'database', 'open', 'emanating', 'shutdown', 'initialization', 'oracle', 'kept', 'returning', 'ora', 'internal', 'code', 'arguments', 'proceed', 'database', 'rdbms', 'ordbms', 'database', 'rdbms', 'ordbms']


In [8]:
processed_docs = np.array([preprocess(x) for x in data])
del data

In [9]:
bigram = gensim.models.Phrases(processed_docs, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[processed_docs], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [10]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags and not token.lemma_ in my_stop_words])
    return texts_out

In [ ]:
processed_docs = make_bigrams(processed_docs)
processed_docs = make_trigrams(processed_docs)

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

processed_docs = lemmatization(processed_docs, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=300,
                                           random_state=100,
                                           update_every=2,
                                           chunksize=50000,
                                           passes=15)

print('\nPerplexity: ', lda_model.log_perplexity(bow_corpus))

coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
lda_model.save(datapath(base_model_lda + "model_semi_final"))

In [ ]:
def get_text_bow(text):
    text = preprocess(text)
    text = make_trigrams([text])[0]
    text = lemmatization([text])[0]
    bow_vector = dictionary.doc2bow(text)
    return bow_vector

def test_texts(text1, text2):
    bow1 = get_text_bow(text1)
    bow2 = get_text_bow(text2)
    for index, score in sorted(lda_model[bow1], key=lambda tup: -1*tup[1]):
        print(f"index: {index}, score {score}")
        #print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))
    print("_________________________________________")
    for index, score in sorted(lda_model[bow2], key=lambda tup: -1*tup[1]):
        print(f"index: {index}, score {score}")

In [ ]:
data = pd.read_csv(base_dataset + 'large_data.csv')
data = data.head(DOC_COUNT)

In [ ]:
row_n = 765
test_texts(data.loc[row_n, 'post_title'], data.loc[row_n, 'post_body'])
print("______________________________")
print(data.loc[row_n, 'post_title'])
print("______________________________")
print(data.loc[row_n, 'post_body'])